In [ ]:
from nbdev import *
%nbdev_default_export callbacks

Cells will be exported to deepflash2.callbacks,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Callbacks

> Implements utilities called at certain points during model training. See the [fastai2 docs](https://dev.fast.ai/callback.core#Callback) for more information.

In [ ]:
%nbdev_hide
from nbdev.showdoc import *

In [ ]:
%nbdev_export
from fastai.callback.core import Callback

## Recomputes elastic deformations after each epoch

In [ ]:
%nbdev_export
class ElasticDeformCallback(Callback):
    "`Callback` that recomputes elastic deformations after each epoch"
    run_valid = False
    def after_epoch(self):
        "Randomly recompute elastic deformations"
        self.learn.dls.train.on_epoch_end()

### Example


In [ ]:
#learn = Learner(dls, model, cbs=ElasticDeformCallback)

## Export -

In [ ]:
%nbdev_hide
from nbdev.export import *
notebook2script()

Converted 00_learner.ipynb.
Converted 01_models.ipynb.
Converted 02_data.ipynb.
Converted 03_metrics.ipynb.
Converted 04_callbacks.ipynb.
Converted 05_losses.ipynb.
Converted 06_utils.ipynb.
Converted gt_estimation.ipynb.
Converted index.ipynb.
